# Підготовка даних

## Файл csv із збігами та їх прізвищами

Мають бути такі колонки

- surname - прізвище предка збіженця. У чоловічому роді, українською мовою
- name - ім'я збіженця
- cm - число, спільна днк в сантиморганах
- url - покликання на збіженця

# Параметри

In [2]:
# Всі необхідні файли потрібно покласи у цей каталог
directory = "data"

# Файл із збіженцями та прізвищами їх предків
input_matches = "matches.csv"

# Відкриваємо вхідний файл із збіженцями прізвищами їх предків та базу даних

In [8]:
import sqlite3

db = sqlite3.connect(f'{directory}/surnames.db')

In [6]:
import pandas as pd
matches_df = pd.read_csv(f'{directory}/matches.csv')

In [5]:
matches_df.groupby("url")

matches = {}
for row in matches_df.itertuples(index=True, name='Pandas'):
    print(getattr(row, "Name"), getattr(row, "Percentage"))

,surname,group,name,cm,url
0,Шиліпук,NaN,Olya Shylipuk,41.7,https://www.myheritage.com.ua/dna/match/D-FE4A...
1,Барчук,NaN,Olya Shylipuk,41.7,https://www.myheritage.com.ua/dna/match/D-FE4A...
2,Шмаленко,NaN,Inna Shmalenko,40.5,https://www.myheritage.com.ua/dna/match/D-FE4A...
3,Купчик,NaN,Inna Shmalenko,40.5,https://www.myheritage.com.ua/dna/match/D-FE4A...
4,Полікарпенко,NaN,Александр Поликаренко,37.7,https://www.myheritage.com.ua/dna/match/D-FE4A...
...,...,...,...,...,...
102,Лащук,NaN,John Michalchuk,24.2,https://www.myheritage.com.ua/dna/match/D-FE4A...
103,Буз,M,Natalia Kardash,24.1,https://www.myheritage.com.ua/dna/match/D-FE4A...
104,Солодуха,M,Natalia Kardash,24.1,https://www.myheritage.com.ua/dna/match/D-FE4A...
105,Оселедько,NaN,Oksana Baškė,24.1,https://www.myheritage.com.ua/dna/match/D-FE4A...


In [9]:
matches_df.groupby("url")